<a href="https://colab.research.google.com/github/shhuangmust/AI/blob/111-2/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝需要的套件

In [1]:
!pip install langchain
!pip install openai
!pip install google-search-results
!pip install unstructured
!pip install 'chromadb>=0.3.5'
!pip install pinecone-client
!pip install youtube-transcript-api
!pip install pytube
!pip install pdf2image
!pip install pytesseract
!pip install tiktoken

# Python要3.9以上

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for

In [2]:
# 先去https://serpapi.com/申請API KEY

import os
os.environ["OPENAI_API_KEY"] = "你自己的Openai api key"
os.environ["SERPAPI_API_KEY"] = "你自己的Serp api key"

apikey = os.environ["OPENAI_API_KEY"]
serpapikey = os.environ["SERPAPI_API_KEY"]

In [3]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003",max_tokens=1024)
llm("如何評價人工智慧？")

'\n\n人工智慧的評價，可以從多個方面來考慮，如：\n\n1. 精確度：可以衡量人工智慧系統在應用場景中的正確性，以及它的准確性和可靠性。\n\n2. 效率：可以衡量人工智慧系統在應用場景中的運行效率，以及它的計算成本和時間成本。\n\n3. 可用性：可以衡量人工智慧系統的可用性，以及它的易用性和穩定性。\n\n4. 可擴展性：可以衡量人工智慧系統的可擴展性，以及它可以支持的功能的數量和品質。\n\n5. 靈活性：可以衡量人工智慧系統的靈活性，以及它能夠應對不同應用場景的能力。'

我們可以看到，他正確的返回了日期（有時差），並且返回了歷史上的今天。

在鏈上和代理上都會有verbose這個參數，這個是非常有用的參數，啟動他們之後我們可以看到完整的鏈執行過程。

可以在上面返回的結果看到，他們將我們的問題分解了幾個步驟，然後一步一步得到最終的答案。

關於代理類型幾個選項的含義（理解不了也不會影響下面的學習，用多了自然理解了）：

* zero-shot-react-description: 根據工具的描述和請求內容的來決定使用哪個工具（最常用）
* react-docstore: 使用 ReAct 框架和 docstore 交互, 使用Search和Lookup工具, 前者使用來搜索, 後者尋找術語, 舉例:Wipipedia工具
* self-ask-with-search 這種代理只使用一個工具: Intermediate Answer, 它會為問題尋找答案(指的非gpt生成的答案, 而在網絡中,文件中已經存在的), 如Google search API工具

conversational-react-description: 為會議設置和設計的代理，它的提示會議被設計的工具具有會議性質，而且還是會使用 ReAct 框架來決定使用來個工具，並且將過往的會話交互存入記憶體

reAct 介紹可以看這個：https://arxiv.org/pdf/2210.03629.pdf

LLM 的 ReAct 模式的 Python 實踐：https ://til.simonwillison.net/llms/python-react-pattern

代理類型官方解釋：

https://python.langchain.com/en/latest/modules/agents/agents/agent\_types.html?highlight=zero-shot-react-description

有一點要說明的是，這個serpapi貌似對中文不是很好，所以提出問題的提示建議使用英文。

當然，官方已經寫好了ChatGPT Plugins的agent，未來chatgpt能用啥插件，我們在api裡面也能用插件，想都美滋滋。

不要過目前只能使用不使用授權的插件，未來等待官方解決這個問題。

### 對超長文字進行總結

如果我們想用openai api對一段文件進行總結，我們通常的做法就是直接發給api讓他總結。但是如果文件超過了api最大的token限制就會報錯。

這個時候，我們一般會進行對文章進行分段，比通過tiktoken計算並分段，然後將各段發送給api 進行總結，最後將各一段的總結再進行一個全部的總結。

如果，你用的是LangChain，他很好的幫我們處理了這個過程，使我們編寫了程式碼變的非常簡單。

In [4]:
!wget https://github.com/shhuangmust/AI/raw/111-2/must.txt

--2023-05-18 14:26:33--  https://github.com/shhuangmust/AI/raw/111-2/must.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/111-2/must.txt [following]
--2023-05-18 14:26:33--  https://raw.githubusercontent.com/shhuangmust/AI/111-2/must.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26674 (26K) [text/plain]
Saving to: ‘must.txt’

must.txt            100%[===================>]  26.05K  --.-KB/s    in 0s      

2023-05-18 14:26:33 (130 MB/s) - ‘must.txt’ saved [26674/26674]



In [5]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import TextLoader


# 導入文字
# loader = UnstructuredFileLoader("lg_test.txt")
loader = TextLoader("must.txt")

# 將文字轉換成Document物件
document = loader.load()
print(f'documents:{len(document)}')

# 初始化文字分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

# 切分文字
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

documents:1
documents:32


In [6]:
llm = OpenAI(max_tokens=1500)

# 建立總結chain
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

In [7]:
# 執行總結的chain（只取前5段）
chain.run(split_documents[:5])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"中文維基百科Facebook粉絲專頁正式上線，邀請大家一同關注。
[關閉]
明新科技大學
維基百科，自由的百科全書"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point:  中文維基百科的Facebook粉絲專頁正式上線，歡迎大家一同關注，該百科全書是一個自由的百科全書。
We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
本條目存在以下問題，請協助改善本條目或在討論頁針對議題發表看法。
此條目疑似為廣告或包含宣傳性內容。 (2019年3月19日)
請協助使用中立的觀點重寫這篇條目。明顯的廣告請加入{{Delete|G11}}來提請刪除。
此條目包含過多僅特定讀者會感興趣的過度細節內容。 (2019年3月19日)
請重新整理本條目以切合主題，並移除與維基百科內容方針相悖的過度細節內容。詳細資訊請參見討論頁。
Confusion grey.svg  提示：此條目的主題不是明志科技大學。
明新學校財團法人
明新科技大學
Minghsin University of Science and Technology logo.svg
明新科技大學校門外一景.jpg
其他名字	Minghsin University of Science and Technology
校訓	堅毅、求新、創造
創辦時間	1966年成立，時名明新工業專科學校
1997年改制，時名明新技術學院
2

'\n\n中文維基百科的Facebook粉絲專頁正式上線，歡迎大家一同關注，該百科全書記載了關於新竹縣新豐鄉明新科技大學的詳細資料，學生人數為10,556（2022年），校址為新興路1號，總面積為32公頃，網站為must.edu.tw，該校於1966年創立，擁有5個學院、20個學系、9個研究所，位於湖口台地，毗鄰中山高速公路、新豐火車站、新竹工業園區及台元科技園區，距新竹科學園區僅八公里，鐵路及公路交通皆四通八達，該校自成立以來不斷增設學系及學位學程，包括學士後行銷與流通管理學士學位學程、學士後旅館管理實務學士學位學程、化妝品應用學士學位學程、文化觀光產業學士學位學程、創業與行銷學士學位學程、雲端科技與商務應用學士學位學程、多媒體與遊戲發展系、時尚造型與設計系等，請協助使用中立的觀點重寫這篇條目及移除與維基百科內容方針相悖的過度細節內容，以確保該百科全書內容的質量維持高水準。\n\n中文維基百科的Facebook粉絲專頁正式上線，歡迎大家一同關注，該百科全書記載了新竹縣新豐鄉明新科技大學的詳細資料，學生人數為10,556（2022年），校址為新興路1號，總面積為32公頃，網站為must.edu.tw，該校於1966年創立，擁有5個學院、20個學系、9個研究所，及從學士後行銷與流通管理學士學位學程、學士後旅館管理實務學士學位學程、化妝品應用學士學位學程、文化觀光產業學士學位學程、創業與行銷學士學位學程、雲端科技與商務應用學士學位學程、多媒體與遊戲發展系、時尚造型與設計系等等，位於湖口台地，毗鄰中山高速公路、新豐火車站、新竹工業園區及台元科技園區，距新竹科學園區僅八公里，鐵路及公路交通皆四通八達，為確保百科內容的質量維持高水準，請協助使用中立的'

In [8]:
!pip install 'chromadb>=0.3.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 建立本機知識庫QA機器人

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

# 載入資料夾中所有TXT檔案
loader = DirectoryLoader('./', glob='**/*.txt')

# 將資料轉成document物佚，每個檔案會為作為一個document
documents = loader.load()

# 初始化載入器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 物件
embeddings = OpenAIEmbeddings()

# 將 document 透過 openai 的 embeddings 物件計算 embedding向量資料暫時存入 Chroma 向量資料庫用於後續的搜尋
docsearch = Chroma.from_documents(split_docs, embeddings)

# 建立回答物件
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

# 進行回答
# result = qa({"query": "現行明新科技大學之校訓為何？"})
result = qa({"query": "工專時期第3任校長是誰"})
print(result['result'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:201: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


 林世明


In [10]:
print(result)

{'query': '工專時期第3任校長是誰', 'result': ' 林世明', 'source_documents': [Document(page_content='歷任校長\n\n工專時期\n\n任\t姓名\t任期\n\n第一任\t李鴻超\t1966年\n\n第二任\t陳天志\t1990年\n\n第三任\t林世明\t不詳\n\n技術學院時期\n\n任\t姓名\t任期', metadata={'source': 'must.txt'}), Document(page_content='代理校長\t李小超\t2007年4月－2007年7月\n\n第三任\t楊肇政\t2007年8月－2010年7月\n\n第四任\t馮丹白\t2010年8月－2013年7月\n\n第五任\t袁保新\t2013年8月－2017年8月', metadata={'source': 'must.txt'}), Document(page_content='代理校長\t張哲郎\t2005年2月－2005年6月\n\n代理校長\t葉玉莉\t2005年6月－2005年12月\n\n第二任\t林世明\t2005年12月－2007年3月', metadata={'source': 'must.txt'}), Document(page_content='第一任\t林世明\t1997年8月－2001年7月\n\n第二任\t張光正\t2001年8月－2002年8月\n\n科技大學時期\n\n任\t姓名\t任期\n\n第一任\t張光正\t2002年9月－2005年1月', metadata={'source': 'must.txt'})]}


## 持久化embedding資料庫

In [11]:
docsearch = Chroma.from_documents(split_docs, embeddings, persist_directory="./vector_store")
docsearch.persist()

In [12]:
docsearch = Chroma(persist_directory="./vector_store", embedding_function=embeddings)
result = qa({"query": "工專時期第二任校長為何？"})
print(result['result'])

 陳天志


In [13]:
result = qa({"query": "校歌中，同建三民主義新中國的下一句是什麼？"})
print(result['result'])

 光芒普照太平洋!
